In [58]:
#Importar las librerias necesarias
import geopandas as gpd
from copy import deepcopy
import os, math
import numpy as np
import requests
import pandas as pd
import gdown
from urllib.parse import urlparse
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import time
import zipfile
import warnings
import os
import tempfile
import shutil
from selenium.webdriver.chrome.options import Options

In [59]:
#Establecer directorio temporal de trabajo
os.create_tempdir = True
if os.create_tempdir:
    temp_dir = os.path.join(os.getcwd(), "temp")
    if not os.path.exists(temp_dir):
        os.makedirs(temp_dir)
    os.chdir(temp_dir)


In [60]:
# Descargar y descomprimir el chromedriver
url = "https://storage.googleapis.com/chrome-for-testing-public/137.0.7151.68/win64/chromedriver-win64.zip" #Esta ruta cambia dependiendo de tu versión de google chrome
zip_path = "chromedriver-win64.zip"
extract_path = "chromedriver-win64"

response = requests.get(url)
with open(zip_path, "wb") as f:
    f.write(response.content)
    
if zipfile.is_zipfile(zip_path):
    with zipfile.ZipFile(zip_path, "r") as zip_ref:
        zip_ref.extractall(extract_path)
    os.remove(zip_path) 

webdriver_path = r'chromedriver-win64\chromedriver-win64\chromedriver.exe'
options = webdriver.ChromeOptions()
options.add_argument('headless=False')
options.add_argument('--disable-gpu')
service = Service(webdriver_path)
driver = webdriver.Chrome(service=service, options=options)

In [87]:
# Crear un directorio temporal
descarga_dir = tempfile.mkdtemp()

# Configurar Chrome para permitir descargas sin preguntar
chrome_options = Options()
prefs = {
    "download.default_directory": descarga_dir,
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "safebrowsing.enabled": True,
    "profile.default_content_setting_values.automatic_downloads": 1,
    "profile.content_settings.exceptions.automatic_downloads.*.setting": 1
}
chrome_options.add_experimental_option("prefs", prefs)

# Inicializar el navegador
driver = webdriver.Chrome(options=chrome_options)

# Abrir la URL
url = "http://dgeiawf.semarnat.gob.mx:8080/ibi_apps/WFServlet?IBIF_ex=D3_AGUA07_09%26IBIC_user=dgeia_mce%26IBIC_pass=dgeia_mce&NOMBREANIO=*&NOMBREENTIDAD=Ciudad%20de%20M%E9xico"
driver.get(url)

# Esperar a que cargue y dar clic al botón de Excel
driver.implicitly_wait(10)
driver.find_element(By.CSS_SELECTOR, 'input[title="Reporte Excel"]').click()

# Esperar a que se descargue el archivo (espera activa hasta que termine la descarga)
archivo_descargado = None
tiempo_max_espera = 60
inicio = time.time()
while time.time() - inicio < tiempo_max_espera:
    archivos = os.listdir(descarga_dir)
    xls_files = [f for f in archivos if f.endswith(".xlsx") or f.endswith(".xls")]
    time.sleep(1)
# Cerrar el navegador
driver.quit()


In [88]:
print("Contenido de descarga_dir:")
for archivo in os.listdir(descarga_dir):
    ruta_completa = os.path.join(descarga_dir, archivo)
    if os.path.isdir(ruta_completa):
        tipo = "Directorio"
    elif os.path.isfile(ruta_completa):
        # Obtener extensión del archivo
        _, extension = os.path.splitext(archivo)
        tipo = f"Archivo {extension}" if extension else "Archivo sin extensión"
    else:
        tipo = "Otro tipo"
    print(f"{archivo} - {tipo}")


Contenido de descarga_dir:
xls - Archivo sin extensión


In [ ]:
ruta_completa = os.path.join(descarga_dir, "xls")
print("Ruta completa del archivo:", ruta_completa)

In [ ]:
!pip install --upgrade xlrd

In [97]:
#Leer el archivo Excel descargado
df = pd.read_excel(ruta_completa, engine='xlrd')
print(df.head())

ImportError: Pandas requires version '2.0.1' or newer of 'xlrd' (version '1.2.0' currently installed).